# Python Environment
시스템 내 기본 python과 jupyter notebook에서 사용하는 python이 같은 지 확인하는 법
> Cell 내 윗 주석은 설명, 아래 주석은 찬영의 결과

In [ ]:
# 앞에 느낌표를 붙이면 터미널 명령어(Command Prompt에 명령어 입력)로써 작동하는데,
# 컴퓨터 내 기본 python이 무엇인지 찾기 위해서 살펴본다.
!python --version

# "Python 3.6.8"

In [ ]:
# python이 여러 개가 다운로드 된 경우 여러 개가 등장한다.
!WHERE python

# "C:\Users\codybright\AppData\Local\Programs\Python\Python36\python.exe"

In [ ]:
# jupyter notebook이 커널로써 사용하는 python
import sys
sys.executable

# "c:\\users\\codybright\\appdata\\local\\programs\\python\\python36\\python.exe"

## 프로그램이 실행되는 원리

`(설명을 위해 하는 설명이라 정확한 개념은 아니다)`

- 프로그램이 실행되는 것은 쉽게 말하면 program.exe가 실행되는 것.
- 그래서 우리가 Command Prompt에서 python을 입력하면 python이 실행되는 것이다.

<img src="./public/python.PNG" align="left" width="700px"/>

### 그렇다면, 컴퓨터는 이러한 .exe 파일을 어디 있는 줄 알고 똑똑하게 실행할까? 
- 프로그램을 둘러볼 위치를 정의한 것이 바로 ***환경 변수***
- 환경 변수는 시스템 변수와 사용자 변수가 있는데, 뭐 둘다 둘러볼 위치이고,
- 보통 python.exe의 위치가 환경 변수 내 PATH에 저장되어 있어서 실행되는 것이다.

> 그렇다면, 코드로 환경 변수 리스트를 확인해보자.

In [ ]:
import os
os.environ["PATH"].split(";")

필자의 경우 python과 관련된 부분은 아래와 같고, 결론적으로 jupyter notebook에서 쓰는 그리고 시스템 기본 python이 같다.

- C:\\Users\\codybright\\AppData\\Local\\Programs\\Python\\Python36\\Scripts\\
- C:\\Users\\codybright\\AppData\\Local\\Programs\\Python\\Python36\\
- c:\\users\\codybright\\appdata\\local\\programs\\python\\python36\\lib\\site-packages\\pywin32_system32


# torch Environment

필자도 예시 코드를 바탕으로 코드를 만드는데, 모듈 import하는 부분부터 오류가 있었다.
<img src="./public/error.PNG" align="left" width="500px"/>

오류를 살펴보니, torch는 있지만, torch.utils.tensorboard은 없는 것이다.

> 왜냐하면, 필자는 torch v1.0.1을 쓰는데, tensorboard는 구글 검색 결과 1.3.0부터 등장하는 듯 했다.

어렵게 생각할 것 없이 업데이트를 해주면 된다.

***하지만, 업데이트와 더불어 설치된 모듈 정보도 확인해보자.***

In [ ]:
# pip이 온전히 설치된 사람은 결과가 등장할 것이다.
# 필자는 순정 python, 순정 pip을 설치하였기에 터미널에서 pip이 동작한다.
# pip도 pip.exe와 같이 실행되는 것이다.

!pip show torch

In [ ]:
!WHERE pip

### 업그레이드의 경우 보통 검색해보면,

`pip install torch` or `pip install --upgrade torch torchvision`

즉, pip이 설치 명령어만 줘도 알아서 업그레이드 해준다 or 업그레이드 조건을 준다인데,

사실상 여러 버전이 깔리는 것을 원치 않는다면, 삭제해주고 설치하는 것이 낫다.

그래서 필자는

`pip uninstall torch torchvision`

`pip install torch==1.5.0+cpu torchvision==0.6.0+cpu -f https://download.pytorch.org/whl/torch_stable.html`

을 실행했다.

그냥 `pip install torch torchvision`을 하니 오류가 났었다.

# Face Identification

안면 인식 관련하여 크게, Face Detection과 Face Identification으로 나뉜다. 둘은 비슷하지만 다르다.

## Import Libraries

현재 이 코드가 적힌 파일 위치에 facenet_pytorch이라는 폴더가 있고, 이를 모듈처럼 불러와 사용하는데,

이것도 `__init__.py`라는 파일이 있어야 이렇게 모듈처럼 불러올 수 있고, 동일 폴더 내에 있어야 하는 등 여러 조건이 있다.

In [1]:
# 구글이 공개한 facenet이라는 AI 모델이 있는데, tensorflow로 작성된 것을 pytorch 형태로 공개하고 있다.
# https://github.com/timesler/facenet-pytorch
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training

In [2]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os

## Import the Data

우리가 봤었던 MNIST의 경우, 데이터가 이미지인데 excel에 숫자로 적혀 있어서 pandas로 불러와서 이를 torch로 바꿔주었으나,

`featuresTrain = torch.from_numpy(features_train)`

`targetsTrain = torch.from_numpy(target_train).type(torch.LongTensor)`

`train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)`

여기서는 이미지 파일이기 때문에 torchvision.datasets이라는 내장된 모듈을 사용해 간편하게 불러온다.

그리고 한번 이미지를 align 해야 하는 과정이 필요하다.

`dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))`

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

보통 input data 위치 및 하이퍼파라미터를 설정하는 것인데, 하이퍼 파라미터란 조정하는 수치값을 의미한다.

epoch의 경우 같은 데이터로 학습을 몇 번 돌릴 것인지를 의미한다.

In [3]:
data_dir = './data/train'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 4

#### Determine if an nvidia GPU is available

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtained cropped faces.

In [6]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_aligned')) for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 2 of 2

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [7]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [8]:
len(dataset.class_to_idx)

6

#### Define optimizer, scheduler, dataset, and dataloader

In [9]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_aligned', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [21]:
len(img_inds)

49

#### Define loss and evaluation functions

In [10]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [11]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     1/1    | loss:    1.8074 | fps:    6.4688 | acc:    0.2000   

Epoch 1/8
----------
Train |     2/2    | loss:    1.1343 | fps:    5.5490 | acc:    0.4509   
Valid |     1/1    | loss:    5.0978 | fps:   20.5402 | acc:    0.1000   

Epoch 2/8
----------
Train |     2/2    | loss:    0.3976 | fps:    5.4350 | acc:    0.8906   
Valid |     1/1    | loss:    2.2830 | fps:   20.5447 | acc:    0.3000   

Epoch 3/8
----------
Train |     2/2    | loss:    0.1105 | fps:    5.4670 | acc:    1.0000   
Valid |     1/1    | loss:    2.6657 | fps:   20.0937 | acc:    0.4000   

Epoch 4/8
----------
Train |     2/2    | loss:    0.0396 | fps:    5.5004 | acc:    0.9844   
Valid |     1/1    | loss:    4.4182 | fps:   20.6519 | acc:    0.3000   

Epoch 5/8
----------
Train |     2/2    | loss:    0.2939 | fps:    5.5030 | acc:    0.7545   
Valid |     1/1    | loss:    6.2994 | fps:   20.7455 | acc:    0.2000   

Epoch 6/8
----------
Train |     2/2    | loss:    0.2

In [24]:
data_loader = DataLoader(dataset)

In [46]:
X = []
y = []
for i in data_loader:
    X += resnet(i[0]).tolist()
    y.append(i[1].tolist())

In [33]:
from sklearn import svm

In [50]:
clf = svm.SVC()
clf.fit(X, y)

c:\users\codybright\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\codybright\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [51]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [52]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('./data/test')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [53]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999673
Face detected with probability: 0.999907
Face detected with probability: 0.999785
Face detected with probability: 0.993947
Face detected with probability: 1.000000
Face detected with probability: 0.999940
Face detected with probability: 0.999914
Face detected with probability: 0.999175
Face detected with probability: 0.999949
Face detected with probability: 0.999987
Face detected with probability: 0.999748
Face detected with probability: 0.999990
Face detected with probability: 0.999611
Face detected with probability: 0.999680
Face detected with probability: 0.999252
Face detected with probability: 0.999713
Face detected with probability: 0.999566
Face detected with probability: 0.999360
Face detected with probability: 0.999735
Face detected with probability: 0.999720
Face detected with probability: 0.999988
Face detected with probability: 0.999891
Face detected with probability: 0.999953
Face detected with probability: 0.999914


In [54]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [57]:
embeddings.tolist()

[[5.078503608703613,
  -2.264317035675049,
  -4.872681617736816,
  -1.73228919506073,
  -1.395796775817871,
  5.50147008895874],
 [8.396160125732422,
  0.14067482948303223,
  -4.982182025909424,
  -4.455480098724365,
  -0.7572050094604492,
  2.6791553497314453],
 [4.576371192932129,
  -3.0168094635009766,
  -1.2595305442810059,
  -4.206353664398193,
  0.3566843271255493,
  3.353727340698242],
 [9.80397891998291,
  0.1862918734550476,
  -4.759002685546875,
  -4.237934589385986,
  -0.9220809936523438,
  0.6107087135314941],
 [-1.1356827020645142,
  -2.675570487976074,
  0.5298291444778442,
  -1.235497236251831,
  3.673877716064453,
  -0.4325626790523529],
 [2.8974320888519287,
  0.7188677191734314,
  -2.433302402496338,
  -1.74935781955719,
  0.37100690603256226,
  0.3533100485801697],
 [5.403720855712891,
  2.1207809448242188,
  -2.644862174987793,
  -2.424196720123291,
  -1.7654244899749756,
  -0.7605353593826294],
 [2.2574615478515625,
  0.8495261669158936,
  -2.4450864791870117,
  -2

In [60]:
print(clf.predict(embeddings.tolist()))
print(names)
print(dataset.idx_to_class)

[1 0 1 0 1 1 1 1 1 2 2 2 3 4 3 3 0 1 1 4 4 1 1 1]
['IU', 'IU', 'IU', 'IU', 'eunwoo', 'eunwoo', 'eunwoo', 'eunwoo', 'gaeri', 'gaeri', 'gaeri', 'gaeri', 'jonggook', 'jonggook', 'jonggook', 'jonggook', 'soohyang', 'soohyang', 'soohyang', 'soohyang', 'younha', 'younha', 'younha', 'younha']
{0: 'IU', 1: 'eunwoo', 2: 'gaeri', 3: 'jonggook', 4: 'soohyang', 5: 'younha'}
